## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.14.0               |   py37h89c1867_0        1010 KB  conda-forge
    toolz-0.12.1               |     pyhd8ed1ab_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following NEW packages will be INSTALLED:

  toolz              conda-forge/noarch::toolz-0.12.1-pyhd8ed1ab_0

The following packages will be UPDATED:

  conda                               4.12.0-py37h89c1867_0 --> 4.14.0-py37h89c1867_0



toolz-0.12.1         | 51 KB     | ##################################### | 100% 
conda-4.14.0         | 1010 KB   | ##################################### | 100% 
Preparing transaction: done
Ver

## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(1, 2):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:23<00:00, 43.39it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_1"])
layer_hs_array.shape

(10000, 267264)

## Layer 1 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

# dim_reduced_vecs = layer_hs_array

random_projector = GaussianRandomProjection(random_state = 42)
dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 7894)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 7894D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 0.04 s
Outer iteration 0 / 10
  Iteration 19 (0.77 s, search 0.54 s): objective=4462.36 imbalance=1.061 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (1.55 s, search 1.09 s): objective=4465.33 imbalance=1.066 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 10
  Iteration 19 (2.32 s, search 1.64 s): objective=4465.75 imbalance=1.093 nsplit=0       
Objective improved: keep new clusters
Outer iteration 3 / 10
  Iteration 19 (3.10 s, search 2.18 s): objective=4463.46 imbalance=1.052 nsplit=0       
Outer iteration 4 / 10
  Iteration 19 (3.87 s, search 2.73 s): objective=4453.16 imbalance=1.099 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (4.65 s, search 3.28 s): objective=4466.06 imbalance=1.062 nsplit=0       
Objective improved: keep new clusters
Outer iteration 6 / 10
  Iteration 19 (5.43 s, search 3.83 s): obj

4466.93896484375

In [10]:
kmeans.centroids #cluster centers

array([[ 8.6438498e-03, -5.2703163e-05,  5.0219647e-03, ...,
         2.3666222e-03,  6.3544521e-03,  2.1525482e-02],
       [ 1.0764730e-02, -4.7615021e-03,  9.1849286e-03, ...,
        -7.0727215e-04,  5.3371163e-03,  1.9761207e-02],
       [ 5.8959662e-03, -3.4893807e-03,  7.9619605e-03, ...,
        -3.1775041e-03,  6.8585309e-03,  2.1941399e-02],
       ...,
       [ 8.6442195e-03, -8.6462302e-03,  6.6263271e-03, ...,
        -5.8350442e-03,  8.7771155e-03,  1.7608427e-02],
       [ 8.7475888e-03, -6.8623554e-03,  5.4318439e-03, ...,
        -9.3971426e-04,  4.2084442e-03,  1.9294184e-02],
       [ 1.6025262e-02, -1.7577439e-03,  1.0609017e-02, ...,
         4.0966504e-05,  5.0947806e-03,  1.7251659e-02]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

0.99999994
0.9999999
0.9999999
1.0000001
0.99999994
1.0000001
1.0
0.9999999
1.0000001
1.0


In [12]:
kmeans.obj #inertia at each iteration

array([2612.12719727, 4297.75390625, 4365.48486328, 4397.08935547,
       4419.95898438, 4432.41455078, 4439.82617188, 4445.12939453,
       4448.97460938, 4451.62451172, 4453.51513672, 4455.10693359,
       4456.171875  , 4457.02148438, 4457.79882812, 4458.59326172,
       4459.35302734, 4460.33349609, 4461.39208984, 4462.35546875,
       2760.91967773, 4285.08886719, 4371.3515625 , 4405.09326172,
       4419.56738281, 4427.85888672, 4433.58251953, 4437.29150391,
       4440.88671875, 4445.85400391, 4450.93017578, 4455.05273438,
       4458.67041016, 4461.00244141, 4462.48095703, 4463.43017578,
       4464.18310547, 4464.70019531, 4465.06103516, 4465.33300781,
       2581.35449219, 4294.00927734, 4371.8125    , 4401.55273438,
       4419.30126953, 4436.41113281, 4447.36962891, 4451.79638672,
       4454.53173828, 4456.57324219, 4458.3359375 , 4459.66308594,
       4460.90722656, 4462.05664062, 4463.01220703, 4463.86181641,
       4464.69482422, 4465.19042969, 4465.51171875, 4465.74511

In [13]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [14]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [15]:
pd.Series(classifications).value_counts()

0    1325
6    1292
8    1189
3    1179
2    1092
7    1090
1     986
4     818
5     573
9     456
dtype: int64

In [16]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

0    1325
6    1292
8    1189
3    1179
2    1092
7    1090
1     986
4     818
5     573
9     456
dtype: int64

In [17]:
prompt_ids = df["prompt_id"]
prompt_ids = prompt_ids.to_numpy()
prompt_ids

array([ 1,  1,  1, ..., 10, 10, 10])

In [18]:
# Get most common centroid for each 1000 points (same label)
max_centroid_per_label = [pd.Series(classifications[i * 1000:(i + 1) * 1000]).value_counts().idxmax() for i in range(10)]
max_centroid_per_label

[8, 3, 6, 0, 2, 0, 0, 2, 8, 5]

In [19]:
# Get most common label for each point classified to a centroid (same centroid)
centroid_labels = [np.where(classifications == i)[0] for i in range(10)]
max_label_per_centroid = [pd.Series(prompt_ids[centroid_labels[i]]).value_counts().idxmax() for i in range(10)]
max_label_per_centroid

[4, 4, 5, 2, 5, 10, 3, 5, 1, 10]

In [20]:
max_centroids = [centroid for centroid in max_centroid_per_label for _ in range(1000)]

In [21]:
np.array(max_centroids)

array([8, 8, 8, ..., 5, 5, 5])

In [22]:
label_to_centroid = {idx + 1 : max_centroid_per_label[idx] for idx in range(len(max_centroid_per_label))}

centroid_to_label = {idx : max_label_per_centroid[idx] for idx in range(len(max_label_per_centroid))}

In [23]:
label_to_centroid

{1: 8, 2: 3, 3: 6, 4: 0, 5: 2, 6: 0, 7: 0, 8: 2, 9: 8, 10: 5}

In [24]:
centroid_to_label

{0: 4, 1: 4, 2: 5, 3: 2, 4: 5, 5: 10, 6: 3, 7: 5, 8: 1, 9: 10}

In [25]:
vectorized_map = np.vectorize(centroid_to_label.get)
classifications_to_label = vectorized_map(classifications)

classifications_to_label

array([ 5,  4,  1, ..., 10, 10, 10])

In [26]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications_to_label[i * 1000:(i + 1) * 1000] == (i + 1)))

Prompt 1 Accuracy:  0.231
Prompt 2 Accuracy:  0.283
Prompt 3 Accuracy:  0.271
Prompt 4 Accuracy:  0.5
Prompt 5 Accuracy:  0.767
Prompt 6 Accuracy:  0.0
Prompt 7 Accuracy:  0.0
Prompt 8 Accuracy:  0.0
Prompt 9 Accuracy:  0.0
Prompt 10 Accuracy:  0.978


In [27]:
print(f"Overall Accuracy: ", np.mean(classifications_to_label == prompt_ids))

Overall Accuracy:  0.303


In [28]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications[i * 1000:(i + 1) * 1000] == max_centroids[i * 1000:(i + 1)*1000]))

Prompt 1 Accuracy:  0.231
Prompt 2 Accuracy:  0.283
Prompt 3 Accuracy:  0.271
Prompt 4 Accuracy:  0.273
Prompt 5 Accuracy:  0.29
Prompt 6 Accuracy:  0.209
Prompt 7 Accuracy:  0.236
Prompt 8 Accuracy:  0.199
Prompt 9 Accuracy:  0.211
Prompt 10 Accuracy:  0.551


In [29]:
print(f"Overall Accuracy: ", np.mean(classifications == max_centroids))

Overall Accuracy:  0.2754


## Measure Conversion Accuracy

In [30]:
label_to_centroid_vectorized_map = np.vectorize(label_to_centroid.get)
centroid_to_label_vectorized_map = np.vectorize(centroid_to_label.get)
prompt_ids_to_centroids = label_to_centroid_vectorized_map(prompt_ids)
max_centroids_to_labels = centroid_to_label_vectorized_map(max_centroids)

In [31]:
np.mean(max_centroids == prompt_ids_to_centroids) #100% - max_centroids is the prompt ids converted to the max centroid corresponding to each label

1.0

In [32]:
np.mean(max_centroids_to_labels == prompt_ids) #<100%

0.6